In [1]:
import pandas as pd
import numpy as np
import world_bank_data as wb
import polars as pl
from scipy.stats import gmean
import os 
os.chdir('..')

In [2]:
    def adjust(df):
        """
        Function for calculating the adjustment coefficient using the atkinson's method

        Parameters
        ----------
        df : <pd.DataFrame>
            dataframe with the index of the idh index

        Returns
        -------
        <float>
            coefficient of the adjustment
        <float>
            mean of the index
        <float>
            geometric mean of the index
        <float>
            atkinson's coefficient of the index
        """
        gemetric = gmean(df)
        amean = df.mean()
        atkinson = 1 - gemetric/amean
        coef = 1 - atkinson
        return coef, amean, gemetric, atkinson

    def to_category(value):
        mapping = {4:1, 5:2, 6:3, 7:4, 8:5, 
                   9:6, 10:7, 11:8, 2:9, 13:10,
                   14: 11,15: 11, 16:12, 17:12, 
                   18:12.5, 19:13, 20:24, 21:16,
        }
        return mapping.get(value, 0) if value <= 21 else 18

In [3]:
df = pd.DataFrame(wb.get_series('SP.DYN.LE00.IN', country='PR', simplify_index=True))
df = pl.from_pandas(df.reset_index())
df

Year,SP.DYN.LE00.IN
str,f64
"""1960""",68.934
"""1961""",69.262
"""1962""",69.284
"""1963""",69.477
"""1964""",69.693
…,…
"""2019""",79.063
"""2020""",78.041
"""2021""",80.162


In [4]:
df = df.rename({"SP.DYN.LE00.IN":"life_exp"}).fill_null(strategy="forward")
df = df.with_columns(
    pl.col("Year").cast(pl.Int32),
    ((pl.col("life_exp") - 20) / (85-20)).alias("health_index"),
    (((pl.col("life_exp") - 20) / (85-20)) * (1-0.08)).alias("health_index_adjusted"),
    arkinson=0.08
    )
df = df.with_columns(
    (pl.col("health_index").pct_change() * 100).alias("health_index_pct_change"),
    (pl.col("health_index_adjusted").pct_change() * 100).alias("health_index_adjusted_pct_change")
    )
df


Year,life_exp,health_index,health_index_adjusted,arkinson,health_index_pct_change,health_index_adjusted_pct_change
i32,f64,f64,f64,f64,f64,f64
1960,68.934,0.752831,0.692604,0.08,null,null
1961,69.262,0.757877,0.697247,0.08,0.670291,0.670291
1962,69.284,0.758215,0.697558,0.08,0.044659,0.044659
1963,69.477,0.761185,0.70029,0.08,0.391608,0.391608
1964,69.693,0.764508,0.703347,0.08,0.436566,0.436566
…,…,…,…,…,…,…
2019,79.063,0.908662,0.835969,0.08,-1.182868,-1.182868
2020,78.041,0.892938,0.821503,0.08,-1.730356,-1.730356
2021,80.162,0.925569,0.851524,0.08,3.654313,3.654313


In [5]:
# get atlas df from WB (change names)
atlas_df = pl.from_pandas(pd.DataFrame(wb.get_series('NY.GNP.PCAP.PP.CD', country='PR', simplify_index=True).reset_index()))
atlas_df = atlas_df.rename({"NY.GNP.PCAP.PP.CD": "atlas"}).drop_nulls()
atlas_df = atlas_df.with_columns(
    pl.col("Year").cast(pl.Int32),
    pl.col("atlas").cast(pl.Int32)
    )
atlas_df

Year,atlas
i32,i32
1990,9470
1991,9950
1992,10240
1993,10820
1994,11250
…,…
2018,23470
2019,24410
2020,23370


In [6]:
# get gni constant df from WB
gni_df = pl.from_pandas(pd.DataFrame(wb.get_series('NY.GNP.PCAP.PP.KD', country='PR', simplify_index=True).reset_index()))
gni_df = gni_df.rename({'NY.GNP.PCAP.PP.KD': 'gni'})
gni_df = gni_df.with_columns(pl.col("Year").cast(pl.Int32))
atlas_df

Year,atlas
i32,i32
1990,9470
1991,9950
1992,10240
1993,10820
1994,11250
…,…
2018,23470
2019,24410
2020,23370


In [137]:
# adjust values (polars)
adjusted_df = pl.DataFrame({
    "Year": [],
    "coef": [],
    "atkinson": []
})
for file in os.listdir('data/raw/'):
    if file.startswith('data_hpr'):
        adjust_df = pl.read_csv("data/raw/data_hpr_2012_raw.csv")
        adjust_df = adjust_df.select(pl.col("HINCP").drop_nulls())
        adjust_df = adjust_df.sort("HINCP")
        adjust_df = adjust_df.filter(pl.col("HINCP") > 0)

        # replace bottom 0.5% 
        bottom_max = adjust_df.select(pl.col("HINCP").quantile(0.005))
        adjust_df = adjust_df.select(
            pl.when(pl.col("HINCP") < bottom_max)
            .then(bottom_max)
            .otherwise(pl.col("HINCP")).alias("HINCP")
        )

        # drop top 0.5%
        adjust_df = adjust_df.filter(
            pl.col("HINCP") <= pl.col("HINCP").quantile(0.995))
        coef, amean, gemetric, atkinson = adjust(adjust_df)


In [136]:

adjust_df = pl.read_csv("data/raw/data_hpr_2012_raw.csv")
adjust_df = adjust_df.select(pl.col("HINCP").drop_nulls())
adjust_df = adjust_df.sort("HINCP")
adjust_df = adjust_df.filter(pl.col("HINCP") > 0)

# replace bottom 0.5% 
bottom_max = adjust_df.select(pl.col("HINCP").quantile(0.005))
adjust_df = adjust_df.select(
    pl.when(pl.col("HINCP") < bottom_max)
    .then(bottom_max)
    .otherwise(pl.col("HINCP")).alias("HINCP")
)

# drop top 0.5%
adjust_df = adjust_df.filter(
    pl.col("HINCP") <= pl.col("HINCP").quantile(0.995))
coef, amean, gemetric, atkinson = adjust(adjust_df)


0.6211303111584582

In [129]:
adjust_df = pd.read_csv("data/raw/data_hpr_2012_raw.csv")
adjust_df = adjust_df['HINCP'] # use HINCS
adjust_df = adjust_df.sort_values(ascending=True)
adjust_df = adjust_df[adjust_df > 0]
adjust_df = adjust_df.dropna()
bottom_5 = adjust_df[adjust_df <= adjust_df.quantile(0.005)]
bottom_max = bottom_5.max()
# replace bottom 5% with the max value
adjust_df = adjust_df.apply(lambda x: x if x > bottom_max else bottom_max)
# remove top 0.5%
adjust_df = adjust_df[adjust_df <= adjust_df.quantile(0.995)]
adjust_df.reset_index(drop=True, inplace=True)
coef, amean, gemetric, atkinson = adjust(adjust_df)
coef

0.6211757650334937

In [138]:
# adjust values
adjusted_df = pd.DataFrame([], columns=['Year', 'coef', 'atkinson'])
for file in os.listdir('data/raw/'):
    if file.startswith('data_hpr'):
        adjust_df = pd.read_csv('data/raw/' + file)
        adjust_df = adjust_df['HINCP'] # use HINCS
        adjust_df = adjust_df.sort_values(ascending=True)
        adjust_df = adjust_df[adjust_df > 0]
        adjust_df = adjust_df.dropna()
        bottom_5 = adjust_df[adjust_df <= adjust_df.quantile(0.005)]
        bottom_max = bottom_5.max()
        # replace bottom 5% with the max value
        adjust_df = adjust_df.apply(lambda x: x if x > bottom_max else bottom_max)
        # remove top 0.5%
        adjust_df = adjust_df[adjust_df <= adjust_df.quantile(0.995)]
        adjust_df = adjust_df.dropna()
        # get coefficient of ajustment
        coef, amean, gemetric, atkinson = adjust(adjust_df)
        adjusted_df = pd.concat([adjusted_df if not adjusted_df.empty else None,
                                pd.DataFrame([[int(file.split('_')[2]), coef, atkinson]], 
                                                columns=['Year', 'coef', 'atkinson'])])

/tmp/ipykernel_22129/3476718831.py:5: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  adjust_df = pd.read_csv('data/raw/' + file)
/tmp/ipykernel_22129/3476718831.py:5: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  adjust_df = pd.read_csv('data/raw/' + file)
/tmp/ipykernel_22129/3476718831.py:5: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  adjust_df = pd.read_csv('data/raw/' + file)
/tmp/ipykernel_22129/3476718831.py:5: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  adjust_df = pd.read_csv('data/raw/' + file)


In [ ]:
# get atlas df from WB (change names)
atlas_df = pd.DataFrame(wb.get_series('NY.GNP.PCAP.PP.CD', country='PR', simplify_index=True))
atlas_df.reset_index(inplace=True)
atlas_df.rename(columns={'NY.GNP.PCAP.PP.CD': 'atlas'}, inplace=True)
atlas_df['atlas'] = atlas_df['atlas'].astype(float)

# get gni constant df from WB
gni_df = pd.DataFrame(wb.get_series('NY.GNP.PCAP.PP.KD', country='PR', simplify_index=True))
gni_df.reset_index(inplace=True)
gni_df.rename(columns={'NY.GNP.PCAP.PP.KD': 'gni'}, inplace=True)
gni_df['gni'] = gni_df['gni'].astype(float)
# replace value 20

# ajust the index
ajusted_df = pd.DataFrame([], columns=['Year', 'coef', 'atkinson'])
for file in os.listdir('data/raw/'):
    if file.startswith('data_hpr'):
        ajust_df = pd.read_csv('data/raw/' + file, engine="pyarrow")
        ajust_df = ajust_df['HINCP'] # use HINCS
        ajust_df = ajust_df.sort_values(ascending=True)
        ajust_df = ajust_df[ajust_df > 0]
        ajust_df = ajust_df.dropna()
        bottom_5 = ajust_df[ajust_df <= ajust_df.quantile(0.005)]
        bottom_max = bottom_5.max()
        # replace bottom 5% with the max value
        ajust_df = ajust_df.apply(lambda x: x if x > bottom_max else bottom_max)
        # remove top 0.5%
        ajust_df = ajust_df[ajust_df <= ajust_df.quantile(0.995)]
        ajust_df = ajust_df.dropna()
        # get coefficient of ajustment
        coef, amean, gemetric, atkinson = self.adjust(ajust_df)
        ajusted_df = pd.concat([ajusted_df if not ajusted_df.empty else None,
                                pd.DataFrame([[int(file.split('_')[2]), coef, atkinson]], 
                                                columns=['Year', 'coef', 'atkinson'])])
# merge the two dataframes
inc_df = atlas_df.merge(gni_df, on='Year')
inc_df['income_ratio'] = inc_df['gni'] / inc_df['atlas']
inc_df['income_ratio'] = inc_df['income_ratio'].astype(float)
inc_df['Year'] = inc_df['Year'].astype(int)

# merge the income index with the pnb.csv file
pnb = pd.read_csv('data/external/pnb.csv')
merge_df = inc_df.merge(pnb, on='Year', how='left')
merge_df = merge_df.dropna()
merge_df.reset_index(inplace=True)
merge_df.drop(['index'], axis=1, inplace=True)

# calculate the index
merge_df['index'] = (np.log(merge_df['pnb']) - np.log(100)) / (np.log(75000)-np.log(100))
merge_df = merge_df[['Year', 'index']]
merge_df = merge_df.sort_values(by='Year', ascending=True)
merge_df = merge_df.merge(ajusted_df, on='Year', how='left')
merge_df['income_index_ajusted'] = merge_df['coef'] * merge_df['index']
merge_df.drop(['coef'], axis=1, inplace=True)
# growth rate for income index and adjusted income index
merge_df['growth_rate_income_index'] = merge_df['index'].pct_change() * 100
merge_df['growth_rate_income_index_ajusted'] = merge_df['income_index_ajusted'].pct_change() * 100
